In [9]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    
    # Create a dictionary object to hold the json for all patients.
    patients_data = []
        
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        people_dict = self._load_json_file(people_json)
        
        visited_json = kv_data_dir.joinpath('visits.json')
        visited_dict = self._load_json_file(visited_json)
        
        sites_json = kv_data_dir.joinpath('sites.json')
        sites_dict = self._load_json_file(sites_json)
        
        measurements_json = kv_data_dir.joinpath('measurements.json')
        measurements_dict = self._load_json_file(measurements_json)

        self._db_path = Path(db_path)
        self._db = None
        
        # Loop over each patient to add them to the dict; visits will be an array where each visit has a site,and measurements
        # based on that visit.  According to the visit data, there is a NAN value present where the date was not provided.
        for people_key in people_dict:
            visit_id = []
            visits = []
            
            for measure_key in measurements_dict:
                if measurements_dict[measure_key]['person_id'] == people_key:
                    visit_id.append(measurements_dict[measure_key]['visit_id'])  # sets visit id if found
                    
            for visitor_key in visited_dict:
                if visit_id.count(visited_dict[visitor_key]['visit_id']) > 0:
                    visits.append(visited_dict[visitor_key])  # sets all visits
            
            # Set the patient data so far.
            patient_data = (people_dict[people_key])
            patient_data['visits'] = visits
            
            # Loop through the visits to add in the measurements and sites.
            for i, visit in enumerate(patient_data['visits']):
                patient_data['visits'][i]['site'] = sites_dict[visit['site_id']]
                patient_data['visits'][i]['measurements'] = []
                
                for measure_key in measurements_dict:
                    if (measurements_dict[measure_key]['visit_id'] == patient_data['visits'][i]['visit_id']) and (measurements_dict[measure_key]['person_id'] == people_key):
                            patient_data['visits'][i]['measurements'].append(measurements_dict[measure_key])
            
            self.patients_data.append(patient_data)
        
        # Final step for the class
        self._load_db()
        
    # Define a function to load the json files.
    def _load_json_file(self, file_path):
        with open(file_path) as json_file:
            file_contents = json_file.read()
        return json.loads(file_contents)

    def _load_db(self):
        self._db = TinyDB(self._db_path)
        self._db.insert_multiple(self.patients_data)
        self._db.close()

In [10]:
# Final process that puts json data into patient-info.json file.  If it already exists, remove it.
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)